# Your info

Full name: AmirHossein Ahmadi

Student ID: 97522292

# Q1. Hopfield

## Q1.2

In [ ]:
# Q1.2_graded
# Do not change the above line.

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Q1.2_graded
# Do not change the above line.

class HopfieldNet:

    def __init__(self, n):
        self.n = n
        self.weights = np.zeros((n, n))

    def train(self, input):
        input = np.array([input])
        
        p = np.dot(input.T, input)

        np.fill_diagonal(p, 0)
        self.weights += p 

    def is_stable(self, input):
        input = np.array(input)
        dp = np.dot(self.weights, input)

        signs = np.sign(dp)

        return ((signs == input).all(), input, signs)


In [ ]:
# Q1.2_graded
# Do not change the above line.

def print_result(result):
    if (result[0]):
        print(result[1], 'is stable.')
    else:
        print(result[1], 'is unstable. nearest data: ', result[2])

hn = HopfieldNet(6)

hn.train([ 1, 1, 1,-1,-1,-1])
hn.train([ 1,-1, 1,-1, 1,-1])

print_result(hn.is_stable([ 1, 1, 1,-1,-1,-1]))
print_result(hn.is_stable([-1, 1, 1,-1,-1,-1]))

[ 1  1  1 -1 -1 -1] is stable.
[-1  1  1 -1 -1 -1] is unstable. nearest data:  [ 1.  1.  1. -1. -1. -1.]


## Q1.3

In [ ]:
# Q1.3_graded
# Do not change the above line.

from PIL import Image, ImageFont
import os
import numpy as np
import random

In [ ]:
# Q1.3_graded
# Do not change the above line.

path = '/content/CI992/HW3/Q1.3'
chars = 'ABCDEFGHIJ'
fonts = [16, 32, 64]
noise_list = [0.1, 0.3, 0.6]

def generate_data():
    for f in fonts:
        font = ImageFont.truetype(f'{path}/arial.ttf', f)
        for ch in chars:
            im = Image.Image()._new(font.getmask(ch))
            im.save(f'{path}/data/{ch}_{str(f)}.bmp')


def get_noisy_image(img, noise):
    img_arr = np.array(img)

    pixels_num = img_arr.shape[0] * img_arr.shape[1]

    new_image = np.copy(img_arr)
    random_pixels = random.sample(range(0, pixels_num), 
                                  int(pixels_num * noise))

    for pixel in random_pixels:
        row = pixel // img_arr.shape[1]
        col = pixel % img_arr.shape[1]
        
        if img_arr[row][col] > 128:
            new_image[row][col] = 0
        else:
            new_image[row][col] = 255

    new_image = Image.fromarray(new_image, mode='L')
    return new_image


def generate_noisy_data():
    temp = 0
    for noise in noise_list:
        for file_name in os.listdir(f'{path}/data/'):
            if file_name.split('.')[1] != 'bmp':
                continue

            img = Image.open(f'{path}/data/{file_name}', mode='r')
            
            new_image = get_noisy_image(img, noise)
            
            noise_p = int(noise * 100)
            new_image.save(f'{path}/noisy_data/{str(noise_p)}_{file_name}')


generate_data()
generate_noisy_data()

In [ ]:
# Q1.3_graded
# Do not change the above line.

class HopfieldNet:

    def __init__(self, n):
        self.n = n
        self.weights = np.zeros((n, n))

    def train(self, input):
        
        p = np.dot(input.T, input)
        
        np.fill_diagonal(p, 0)
        
        self.weights += p
    
    def restore(self, x):
        p = np.sum(x * self.weights, axis=1)
        return np.array(np.sign(p.T))

    def update(self, input):
        current_x = input
        last_x = np.copy(current_x)
        for i in range(1000):
            current_x = self.restore(current_x)
            if np.array_equal(current_x, last_x):
                break
            last_x = np.copy(current_x)
        return current_x


In [ ]:
# Q1.3_graded
# Do not change the above line.

path = '/content/CI992/HW3/Q1.3'
dim = 40
chars = 'ABCDEFGHIJ'
fonts = [16, 32, 64]
noise_list = [10, 30, 60]

def generate_patterns(font_size):

    patterns = []
    for file_name in os.listdir(f'{path}/data/'):
        if file_name.split('.')[1] != 'bmp' or file_name.split('.')[0].split('_')[1] != str(font_size):
                continue
        
        img = Image.open(f'{path}/data/{file_name}', mode='r')
        img = img.resize((dim, dim))
        img_arr = np.asarray(img, dtype=np.uint8)

        pattern = np.ones(img_arr.shape)
        pattern[img_arr <= 60] = -1

        patterns.append(pattern.flatten())

    return np.array(patterns)


for f in fonts:
    patterns = generate_patterns(f)
    
    hn = HopfieldNet(dim ** 2)
    hn.train(patterns)
    
    for noise in noise_list:
        for ch in chars:
            img = Image.open(f'{path}/noisy_data/{str(noise)}_{ch}_{str(f)}.bmp', mode='r')
            img = img.resize((dim, dim))
            img_arr = np.asarray(img, dtype=np.uint8)

            pattern = np.ones(img_arr.shape)
            pattern[img_arr <= 60] = -1

            result_pattern = hn.update(pattern.flatten())

            result_arr = np.where(result_pattern >= 0, 255, 0)
            result_arr = np.array(result_arr, dtype=np.uint8)

            out = Image.fromarray(result_arr.reshape((dim, dim)), mode='L')
            out.save(f'{path}/result/{str(noise)}_{ch}_{str(f)}.bmp')

            org_img = Image.open(f'{path}/data/{ch}_{str(f)}.bmp', mode='r')
            org_img = org_img.resize((dim, dim))
            org_img_arr = np.asarray(org_img, dtype=np.uint8)

            org_img_pattern = np.ones(org_img_arr.shape)
            org_img_pattern[org_img_arr <= 60] = -1
            org_img_pattern = org_img_pattern.flatten()

            rate = 100 - 100 * np.sum(np.abs(result_pattern - org_img_pattern)) / (2 * dim * dim)
            print(f'Accuracy for noise: {str(noise)}, character: {ch}, font: {str(f)} --> {str(rate)}')

Accuracy for noise: 10, character: A, font: 16 --> 100.0
Accuracy for noise: 10, character: B, font: 16 --> 93.8125
Accuracy for noise: 10, character: C, font: 16 --> 82.875
Accuracy for noise: 10, character: D, font: 16 --> 91.8125
Accuracy for noise: 10, character: E, font: 16 --> 82.75
Accuracy for noise: 10, character: F, font: 16 --> 75.75
Accuracy for noise: 10, character: G, font: 16 --> 76.6875
Accuracy for noise: 10, character: H, font: 16 --> 84.0625
Accuracy for noise: 10, character: I, font: 16 --> 99.375
Accuracy for noise: 10, character: J, font: 16 --> 69.75
Accuracy for noise: 30, character: A, font: 16 --> 51.25
Accuracy for noise: 30, character: B, font: 16 --> 93.8125
Accuracy for noise: 30, character: C, font: 16 --> 82.875
Accuracy for noise: 30, character: D, font: 16 --> 91.8125
Accuracy for noise: 30, character: E, font: 16 --> 82.75
Accuracy for noise: 30, character: F, font: 16 --> 75.75
Accuracy for noise: 30, character: G, font: 16 --> 76.6875
Accuracy for n

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='green'>**D5372R**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
    - **Note**: if you want to add more cells, add this to the **first** line of the cell:
  ```python
  # Q5_graded
  ```
  according to the question number.
  - There are 2 assignments in Gradescope:

    ![assignments](https://i.ibb.co/10GMhGM/Screen-Shot-2021-03-21-at-02-16-25.png)
  
    You should upload your **codes** and your **notebook** in `HW3` section and your final report for all of the questions as a **single pdf** file in `HW3 - Report`. Autograder will automatically check for:
    - `CI992_HW3.ipynb`
    - `Q1.2.py`
    - `Q1.3.py`
    - `inverted_pendulum.fcl`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.